In [1]:
import requests
import pprint
import datetime
from dateutil.relativedelta import relativedelta
import xml.etree.ElementTree as ET
import pandas as pd
import csv
import time

감염자 현황

In [2]:
# 홈피 예시

url = 'http://apis.data.go.kr/1352000/ODMS_COVID_02/callCovid02Api'
key = 'iS0TX2OH/RM+TgSdYsKGiCQUQ7mzMC7NkpBAtqluq8QkSOD8kTywo9cl5zEDn7ade0YEtplGXz40K/ewEl8Pvg=='
dt = '20200120'
param ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '500', 'apiType' : 'xml', 'status_dt' : dt }

res = requests.get(url, params=param)
pprint.pprint(res.content)

(b'<?xml version="1.0" encoding="UTF-8"?>\r\n<response>\r\n <header>\r\n  <re'
 b'sultCode>00</resultCode>\r\n  <resultMsg>NORMAL SERVICE</resultMsg>\r\n </he'
 b'ader>\r\n <body>\r\n  <items>\r\n   <item>\r\n    <accDefRate>25</accDefRate'
 b'>\r\n    <accExamCnt>4</accExamCnt>\r\n    <accExamCompCnt>4</accExamCompCnt'
 b'>\r\n    <careCnt>1</careCnt>\r\n    <dPntCnt>0</dPntCnt>\r\n    <gPntCnt>'
 b'0</gPntCnt>\r\n    <hPntCnt>1</hPntCnt>\r\n    <resutlNegCnt>3</resutlNegCnt'
 b'>\r\n    <statusDt>20200120</statusDt>\r\n    <statusTime>17:00</statusTime>'
 b'\r\n    <uPntCnt>0</uPntCnt>\r\n   </item>\r\n  </items>\r\n  <numOfRows>500'
 b'</numOfRows>\r\n  <pageNo>1</pageNo>\r\n  <totalCount>1196</totalCount>\r'
 b'\n </body>\r\n</response>\r\n\r\n')


In [3]:
# gpt로 만든 xml데이터 가져오기

root = ET.fromstring(res.content)

# 모든 item 요소 가져오기
items = root.findall(".//item")

# 첫 번째 item 요소 선택
item = items[0]

# statusDt와 hPntCnt 값을 가져오기
statusDt = item.findtext("statusDt")
hPntCnt = item.findtext("hPntCnt")

print("statusDt:", statusDt)
print("hPntCnt:", hPntCnt)

statusDt: 20200120
hPntCnt: 1


In [4]:
# datetime으로 월단위 루프 만들기.

dt_time = datetime.datetime(2020, 2, 1)
while dt != '20230301':
    dt_time = dt_time + relativedelta(months=1)
    dt = dt_time.strftime('%Y%m%d')
    print(dt)

20200301
20200401
20200501
20200601
20200701
20200801
20200901
20201001
20201101
20201201
20210101
20210201
20210301
20210401
20210501
20210601
20210701
20210801
20210901
20211001
20211101
20211201
20220101
20220201
20220301
20220401
20220501
20220601
20220701
20220801
20220901
20221001
20221101
20221201
20230101
20230201
20230301


In [5]:
# 두개를 합치기
# 루프 시작 값
url = 'http://apis.data.go.kr/1352000/ODMS_COVID_02/callCovid02Api'
key = 'iS0TX2OH/RM+TgSdYsKGiCQUQ7mzMC7NkpBAtqluq8QkSOD8kTywo9cl5zEDn7ade0YEtplGXz40K/ewEl8Pvg=='
dt = '20200120'
test_list = []
dt_time = datetime.datetime(2020, 3, 1)

# 20230301 완료날자
while dt != '20230301':
    dt_key = dt_time.strftime('%Y년 %m월')
    dt = dt_time.strftime('%Y%m%d')
    
    param ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '500', 'apiType' : 'xml', 'status_dt' : dt }
    
    while True:
        try:
            res = requests.get(url, params=param)
            break
        except requests.exceptions.ConnectionError:
            print('Connection Error occurred. Retrying in 5 seconds...')
            time.sleep(5)
            
    # gpt로 만든 xml데이터 가져오기
    root = ET.fromstring(res.content)

    # 모든 item 요소 가져오기
    items = root.findall(".//item")

    # 첫 번째 item 요소 선택
    item = items[0]
    
    # statusDt와 hPntCnt 값을 가져오기
    hPntCnt = item.findtext("hPntCnt")
    test_list.append(hPntCnt)

    dt_time = dt_time + relativedelta(months=1)

    
test_list.insert(0, '12')
test_list.insert(0, '0')

In [6]:
data = {}
month = []
data_list = []
dt_time = datetime.datetime(2020, 1, 1)
for i in range(len(test_list)-1):
    month.append(dt_time.strftime('%Y년 %m월'))
    data_list.append(int(test_list[i+1]) - int(test_list[i]))
    dt_time = dt_time + relativedelta(months=1)
    
data['기간명'] = month
data['확진자 수'] = data_list

In [7]:
with open('확진자_수.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in data.items():
        writer.writerow([key, value])

In [8]:
df = pd.DataFrame(data)
df

,기간명,확진자 수
0,2020년 01월,12
1,2020년 02월,3514
2,2020년 03월,6361
3,2020년 04월,887
4,2020년 05월,729
5,2020년 06월,1347
6,2020년 07월,1486
7,2020년 08월,5845
8,2020년 09월,3707
9,2020년 10월,2747
